# OpenAI and Ollama integration 🤖

first of all we need to install the needed/required libraries in order to make our solution works

## 1 - install and import

first let's install needed libraries

**ollama** : used to communicate with our local ollama server

**openai** : used to communicate with openai model in the cloud

***N.B:*** we can use openai also to communicate with ollama or another models (like claude) but for training purpose we are going to use both of them

**gradio**: ready to use components developed by huggingface that enable to used modern ui component for ML usage

**python_dotenv**: help load envs variables from the system

In [1]:
pip install ollama python_dotenv openai gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached safehttpx-0.1.6-py3-none-any.whl.metadata (4.2 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 25.3 MB/s eta 0:00:00m eta 0:00:010:01:01
Using cached aiofiles-24.1.0-py3-none-any.whl (15 kB)
Using cached groovy-0.1.2-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 33.7 MB/s eta 0:00:00MB/s eta 0:00:01
Using cached safehttpx-0.1.6-py3-none-any.whl (8.7 kB)
Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.7 MB/s eta

## 2 - import

In [5]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from ollama import chat

after calling the imports we need know to load the envs and start a sample call to open ai first

In [6]:
# Load environment variables from .env file and override existing ones
load_dotenv(override=True)
if os.getenv("OPENAI_API_KEY") is None:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))



before diving into using our first call with our model we need to know how llm model prompts works:
a prompt can be divided into 3 role: system, user, assistant.

**1 - System**:

a system prompt is like a guide to our llm generation that tell it how to behave in each generation for example `you are a helpful doctor assistant, please analyze my symptoms and suggest the best health advice.` here the llm will always behave as a doctor and will see if the generation match the goal of the system prompt and be aligned with it

**2 - User**: 

a user prompt is basically your regular prompt that you entre during you chat with llm in order to have a result for example `i feel tired and sleepy and also i don't have appetite to eat`

**3 - Assistant**:

same as user role but with extra assistance feature like keep providing question in order to assist you or for more clarification

## 3 - let's call the llm📝

first let's make a function that will be used later with gradio 😁

In [8]:
def DocChat(prompt, history):
    messages = [{
        "role": "system",
        "content": "you are a helpful doctor assistant, please analyze my symptoms and suggest the best health advice. without any hillucination and being out of context if you don't know the answer just say you don't know."
    }] + history
    final_prompt = f"""
    this is an medical chat, please analyze the symptoms and suggest the best health advice from user input's
    ---
    {prompt}
    ---
    thanks for you help.
    """
    messages.append({
        "role": "user",
        "content": final_prompt
    })
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        stream=True,
    )
    response = ""
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            yield response

after making this function called `DocChat` let's use gradio chat interface component for testing our work

In [12]:
gr.ChatInterface(DocChat, type="messages", title="Medical Chat Assistant").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## 4 - Ollama🦙 this time 🧐?

In [15]:
def DocChatOllama(propmt, history):
    messages = [{
        "role": "system",
        "content": "you are a helpful doctor assistant, please analyze my symptoms and suggest the best health advice. without any hillucination and being out of context if you don't know the answer just say you don't know."
    }] + history
    final_prompt = f"""
    this is an medical chat, please analyze the symptoms and suggest the best health advice from user input's
    ---
    {propmt}
    ---
    thanks for you help.
    """
    messages.append({
        "role": "user",
        "content": final_prompt
    })
    stream = chat(
        model="gemma3:1b",
        messages=messages,
        stream=True,
    )
    response = ""
    for chunk in stream:
        if chunk['message']['content']:
            response += chunk['message']['content']
            yield response
gr.ChatInterface(DocChatOllama, type="messages", title="Medical Chat Assistant (Ollama)").launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://c00c8753a3144ec12c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
